Source: https://github.com/krishnaik06/Advanced-House-Price-Prediction-/blob/master/Feature%20Engineering.ipynb

In [3]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
# Imports preprocessing functions
%run ./v1_jlu_preprocessing.ipynb

## Functions

#### Algorithms

In [4]:
def predictPrice(X_train, X_test, Y_train, Y_test):
    #regr = SVR().fit(X_train, Y_train)
    regr = Ridge(alpha=100).fit(X_train, Y_train)
    
    train_pred = regr.predict(X_train)
    test_pred = regr.predict(X_test)
    
    print("Train MSE: ", mean_squared_error(Y_train, train_pred))
    print("Test MSE: ", mean_squared_error(Y_test, test_pred))

## Code

In [5]:
begin = time.time()
original_df = pd.read_csv('used_cars_data.csv')
end = time.time()

print("Time taken: ", end - begin)

C:\Users\Adeel\AppData\Local\Temp\ipykernel_12304\1749011710.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv('used_cars_data.csv')


Time taken:  77.09675812721252


In [6]:
# Sample a portion of the rows

begin = time.time()
df = original_df.sample(frac=0.30, replace=False, ignore_index=True)
end = time.time()

print("Time taken: ", end - begin)

print(df.shape)
df.head()

Time taken:  1.4322171211242676
(900012, 66)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,1C6SRFFT5LN235611,45.2 in,NaN,--,67.4 in,Pickup Truck,NaN,Oklahoma City,15.0,NaN,...,A,8-Speed Automatic,t87071,Lone Star Crew Cab 4WD,NaN,4WD,Four-Wheel Drive,144.6 in,82.1 in,2020
1,3TMCZ5AN1JM158792,32.6 in,NaN,--,60.5 in,Pickup Truck,NaN,Ontario,18.0,NaN,...,A,6-Speed Automatic,t74535,TRD Sport Double Cab 4WD,NaN,4WD,Four-Wheel Drive,127.4 in,75.2 in,2018
2,3N1CN8FV2LL818436,31 in,NaN,NaN,NaN,Sedan,NaN,Carrollton,32.0,NaN,...,CVT,Continuously Variable Transmission,t88094,SR FWD,NaN,FWD,Front-Wheel Drive,103.1 in,68.5 in,2020
3,1FMCU9JD2HUC94041,37.3 in,NaN,NaN,NaN,SUV / Crossover,NaN,Fort Wayne,22.0,NaN,...,A,6-Speed Automatic,t66220,Titanium AWD,NaN,AWD,All-Wheel Drive,105.9 in,81.8 in,2017
4,1C6JJTBG1LL163103,38.3 in,NaN,--,60.3 in,Pickup Truck,NaN,Woodbridge,17.0,NaN,...,A,8-Speed Automatic,t84953,Rubicon Crew Cab 4WD,NaN,4WD,Four-Wheel Drive,137.3 in,73.8 in,2020


#### Initial cleaning

In [8]:
df= cleanColumnsAndRows(df)

(900010, 50)


In [9]:
type(df)

pandas.core.frame.DataFrame

#### Process columns with years

In [10]:
df["year"] = processYears(df)
df.head()

,vin,back_legroom,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,...,torque,transmission,transmission_display,trimId,trim_name,wheel_system,wheel_system_display,wheelbase,width,year
0,1C6SRFFT5LN235611,45.2 in,Pickup Truck,Oklahoma City,15.0,276,73132,"$9,813 off MSRP! Priced below KBB Fair Purchas...",V8,5700.0,...,"410 lb-ft @ 3,950 RPM",A,8-Speed Automatic,t87071,Lone Star Crew Cab 4WD,4WD,Four-Wheel Drive,144.6 in,82.1 in,70
1,3TMCZ5AN1JM158792,32.6 in,Pickup Truck,Ontario,18.0,1,97914,2018 Toyota Tacoma TRD Offroad CARFAX One-Owne...,V6,3500.0,...,"265 lb-ft @ 4,600 RPM",A,6-Speed Automatic,t74535,TRD Sport Double Cab 4WD,4WD,Four-Wheel Drive,127.4 in,75.2 in,68
2,3N1CN8FV2LL818436,31 in,Sedan,Carrollton,32.0,347,30117,AUTOMATIC EMERGENCY BRAKING w/PEDESTRIAN DETEC...,I4,1600.0,...,"114 lb-ft @ 4,000 RPM",CVT,Continuously Variable Transmission,t88094,SR FWD,FWD,Front-Wheel Drive,103.1 in,68.5 in,70
3,1FMCU9JD2HUC94041,37.3 in,SUV / Crossover,Fort Wayne,22.0,55,46805,We are open and ready to serve you! Shop safe...,I4,2000.0,...,"275 lb-ft @ 3,000 RPM",A,6-Speed Automatic,t66220,Titanium AWD,AWD,All-Wheel Drive,105.9 in,81.8 in,67
4,1C6JJTBG1LL163103,38.3 in,Pickup Truck,Woodbridge,17.0,274,22191,2020 Jeep Gladiator RubiconPriced below KBB Fa...,V6,3600.0,...,"260 lb-ft @ 4,800 RPM",A,8-Speed Automatic,t84953,Rubicon Crew Cab 4WD,4WD,Four-Wheel Drive,137.3 in,73.8 in,70


In [11]:
df['year']

0         70
1         68
2         70
3         67
4         70
          ..
900005    71
900006    71
900007    64
900008    70
900009    69
Name: year, Length: 900010, dtype: int64

In [12]:
scaler = StandardScaler()

#### Standardize columns of differing data types

In [14]:
df = convertColumnsToNumericalAndStandardize(df)

year


AttributeError: 'tuple' object has no attribute 'head'

#### Run algorithm

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

# predictPrice(X_train, X_test, Y_train, Y_test)

#LinReg Result

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

# predictPrice(X_train, X_test, Y_train, Y_test)

# #SVR Result

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

AttributeError: 'tuple' object has no attribute 'to_numpy'

In [ ]:
# param_grid = {'alpha':[1.5,2,2.5,3,3.5,4,10,20,30,40,50,100], 'solver':['auto', 'svd']}

In [ ]:
grid =  GridSearchCV(predictPrice(X_train, X_test, Y_train, Y_test), param_grid)

In [ ]:
# grid.fit(X_train, Y_train)

In [ ]:
# print(grid.best_params_)
# grid_predictions = grid.predict(X_test)

In [ ]:
predictPrice(X_train, X_test, Y_train, Y_test)